Solución Punto 2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import pairwise_distances
%matplotlib inline

Carga del dataset "Books Ratings"

In [ ]:
df_ratings = pd.read_csv('/content/drive/MyDrive/Proyectos Data Science/Books_rating.csv', sep = ',')

In [ ]:
df_ratings.head(5)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [ ]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Id                  object 
 1   Title               object 
 2   Price               float64
 3   User_id             object 
 4   profileName         object 
 5   review/helpfulness  object 
 6   review/score        float64
 7   review/time         int64  
 8   review/summary      object 
 9   review/text         object 
dtypes: float64(2), int64(1), object(7)
memory usage: 228.9+ MB


Carga del dataset "Books data"

In [ ]:
df_data = pd.read_csv('/content/drive/MyDrive/Proyectos Data Science/books_data.csv', sep = ',')

In [ ]:
df_data.head(5)

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [ ]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212404 entries, 0 to 212403
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Title          212403 non-null  object 
 1   description    143962 non-null  object 
 2   authors        180991 non-null  object 
 3   image          160329 non-null  object 
 4   previewLink    188568 non-null  object 
 5   publisher      136518 non-null  object 
 6   publishedDate  187099 non-null  object 
 7   infoLink       188568 non-null  object 
 8   categories     171205 non-null  object 
 9   ratingsCount   49752 non-null   float64
dtypes: float64(1), object(9)
memory usage: 16.2+ MB


Unimos los dos dataframes mediante la columna en comun que tienen, es decir, el titulo del libro.

In [ ]:
df_books = pd.merge(df_ratings,df_data, on = 'Title')
df_books.shape

(3000000, 19)

In [ ]:
df_books.head(5)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t...",Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN


In [ ]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 19 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Id                  object 
 1   Title               object 
 2   Price               float64
 3   User_id             object 
 4   profileName         object 
 5   review/helpfulness  object 
 6   review/score        float64
 7   review/time         int64  
 8   review/summary      object 
 9   review/text         object 
 10  description         object 
 11  authors             object 
 12  image               object 
 13  previewLink         object 
 14  publisher           object 
 15  publishedDate       object 
 16  infoLink            object 
 17  categories          object 
 18  ratingsCount        float64
dtypes: float64(3), int64(1), object(15)
memory usage: 434.9+ MB


De las 19 columnas que tiene el dataframe no todas son importantes en este caso, así que tomaremos unicamente las columnas: Tittle, price, review/score, review/text, description, authors, categories y ratingsCount.

In [ ]:
df_books_filtered = df_books[['Title','Price','review/score','review/text', 'description','authors','categories','ratingsCount']]

In [ ]:
df_books_filtered

,Title,Price,review/score,review/text,description,authors,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,4.0,This is only for Julie Strain fans. It's a col...,NaN,['Julie Strain'],['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,NaN,5.0,I don't care much for Dr. Seuss but after read...,Philip Nel takes a fascinating look into the k...,['Philip Nel'],['Biography & Autobiography'],NaN
2,Dr. Seuss: American Icon,NaN,5.0,"If people become the books they read and if ""t...",Philip Nel takes a fascinating look into the k...,['Philip Nel'],['Biography & Autobiography'],NaN
3,Dr. Seuss: American Icon,NaN,4.0,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",Philip Nel takes a fascinating look into the k...,['Philip Nel'],['Biography & Autobiography'],NaN
4,Dr. Seuss: American Icon,NaN,4.0,Philip Nel - Dr. Seuss: American IconThis is b...,Philip Nel takes a fascinating look into the k...,['Philip Nel'],['Biography & Autobiography'],NaN
...,...,...,...,...,...,...,...,...
2999995,The Idea of History,NaN,4.0,"This is an extremely difficult book to digest,...","Robin George Collingwood, FBA (1889 - 1943) wa...",['R. G. Collingwood'],['History'],3.0
2999996,The Idea of History,NaN,4.0,This is pretty interesting. Collingwood seems ...,"Robin George Collingwood, FBA (1889 - 1943) wa...",['R. G. Collingwood'],['History'],3.0
2999997,The Idea of History,NaN,4.0,"This is a good book but very esoteric. ""What i...","Robin George Collingwood, FBA (1889 - 1943) wa...",['R. G. Collingwood'],['History'],3.0
2999998,The Idea of History,NaN,5.0,"My daughter, a freshman at Indiana University,...","Robin George Collingwood, FBA (1889 - 1943) wa...",['R. G. Collingwood'],['History'],3.0


Elimanación de duplicados

In [ ]:
df_books_filtered.drop_duplicates(inplace = True)
df_books_filtered.shape

<ipython-input-13-7769370c45e3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books_filtered.drop_duplicates(inplace = True)


(2645903, 8)

Analisis de nulos


In [ ]:
df_books_filtered.isna().sum().sort_values(ascending = False)

Price           2168269
ratingsCount    1245354
description      577052
categories       474198
authors          354248
Title               207
review/text           8
review/score          0
dtype: int64

El titulo es la variable más crucial para el modelo de recomendación, en este caso es mejor eliminar los registros que tienen valores faltantes. Lo mismo haremos para la columna review/text. A las otras columnas que tienen valores nulos se la hara imputación de valores.  

In [ ]:
# Eliminar filas con títulos y review/text faltantes
df_books_filtered = df_books_filtered.dropna(subset=['Title'])
df_books_filtered = df_books_filtered.dropna(subset=['review/text'])

# Imputar valores faltantes
df_books_filtered['Price'] = df_books_filtered['Price'].fillna(df_books_filtered['Price'].median())
df_books_filtered['ratingsCount'] = df_books_filtered['ratingsCount'].fillna(df_books_filtered['ratingsCount'].median())
df_books_filtered['description'] = df_books_filtered['description'].fillna('No description available')
df_books_filtered['categories'] = df_books_filtered['categories'].fillna('Unknown')
df_books_filtered['authors'] = df_books_filtered['authors'].fillna('Unknown')


In [ ]:
df_books_filtered.isna().sum().sort_values(ascending = False)

Title           0
Price           0
review/score    0
review/text     0
description     0
authors         0
categories      0
ratingsCount    0
dtype: int64

Una vez que ya tenemos el dataset organizado podemos hacer una vectorización del contenido: Usaré el TF-IDF para vectorizar las descripciones, categorías de los libros y autores. En este tipo de modelos de recomendación se suelen usar matrices de simulitudes del coseno y se mapean titulos a indices del conjunto de datos.

Dado al tamaño del dataset implementaré NearestNeighbors con paralelización y tomaré una muestra de 10000 libros para poder ejecutar el modelo sin problemas de RAM en colab.

In [ ]:
# Generar el contenido combinado
df_books_filtered['content'] = df_books_filtered['description'] + ' ' + df_books_filtered['categories'] + ' ' + df_books_filtered['authors']

In [ ]:
# Tomar una muestra aleatoria del dataset
sample_df = df_books_filtered.sample(n=10000, random_state=42)  # Tomar una muestra aleatoria de 10000 libros, semilla de 42 para obtener siempre los mismos resultados

In [ ]:
# Receteo del indice para no tener problemas al ejecutar el modelo
sample_df.reset_index(drop=True, inplace=True)
sample_df.head()

,Title,Price,review/score,review/text,description,authors,categories,ratingsCount,content
0,The Giver,14.93,2.0,What a crummy book the Giver is! A novel shoul...,Given his lifetime assignment at the Ceremony ...,['Lois Lowry'],['Juvenile Fiction'],721.0,Given his lifetime assignment at the Ceremony ...
1,Another Roadside Attraction Signed,14.93,5.0,This is my favorite book of all time. I think ...,“Written with a style and humor that haven’t b...,['Tom Robbins'],['Fiction'],33.0,“Written with a style and humor that haven’t b...
2,"Crossroads of Twilight (The Wheel of Time, Boo...",14.93,1.0,"Yes, this is indeed the worst book I have ever...",The Wheel of Time ® is a PBS Great American Re...,['Robert Jordan'],['Fiction'],65.0,The Wheel of Time ® is a PBS Great American Re...
3,archy and mehitabel,14.93,5.0,"Don Marquis created archy the cockroach, who c...",A selection of the best of the hilarious free-...,['Don Marquis'],['Poetry'],1.0,A selection of the best of the hilarious free-...
4,The Killer Angels (Turtleback School & Library...,13.80,5.0,Virtually every seasoned Civil War buff has al...,"Portraits of Lee, Longstreet, and other Civil ...",['Michael Shaara'],['Fiction'],120.0,"Portraits of Lee, Longstreet, and other Civil ..."


In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
train_df, test_df = train_test_split(sample_df, test_size=0.2, random_state=42)
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
# Vectorizar el contenido de los conjuntos de entrenamiento
tfidf = TfidfVectorizer(stop_words='english', max_features=50000)
tfidf_matrix_train = tfidf.fit_transform(train_df['content'])

# Vectorizar el contenido de los conjuntos de prueba usando el mismo vectorizador ajustado en el conjunto de entrenamiento
tfidf_matrix_test = tfidf.transform(test_df['content'])

In [ ]:
# Entrenar el modelo de vecinos más cercanos con los datos de entrenamiento
nbrs_train = NearestNeighbors(n_neighbors=4, metric='cosine', n_jobs=-1).fit(tfidf_matrix_train)

In [ ]:
# Crear una serie para mapear títulos a índices del DataFrame de entrenamiento
indices_train = pd.Series(train_df.index, index=train_df['Title']).drop_duplicates()

In [ ]:
# Definir la función get_recommendations_knn para usar en los datos de prueba
def get_recommendations(title, nbrs_model=nbrs_train, indices=indices_train, tfidf_matrix=tfidf_matrix_train, df=train_df):
    if title not in indices:
        return "Title not found in the training dataset."

    idx = indices[title]
    distances, nbrs_indices = nbrs_model.kneighbors(tfidf_matrix_train[idx], n_neighbors=4)

    # Excluir el propio libro (el más cercano es siempre el libro mismo)
    nbrs_indices = nbrs_indices[0][1:4]

    return df['Title'].iloc[nbrs_indices].values

In [ ]:
# Ejemplo de implementación de la función
book_title = 'Rescue Me'
recommendations = get_recommendations(book_title)
print(f"Libros recomendados basados en '{book_title}':")
for i, title in enumerate(recommendations, 1):
    print(f"{i}. {title}")


Libros recomendados basados en 'Rescue Me':
1. Christopher Blossom: The Greenwich Workshop's New Century Artists Series
2. Golden Days: Memories of a Golden Retriever
3. What Is Love? A Simple Guide to Romantic Happiness


Como no tengo los datos reales de recomendaciones según cada titulo se deben usar otro tipo de métodos para poder analizar el rendimiento del modelo.

In [ ]:
# Función para generar las recomendaciones para un conjunto de títulos
def generate_recommendations(titles, nbrs_model, indices, tfidf_matrix, df):
    recommendations = {}
    for title in titles:
        recommendations[title] = get_recommendations(title, nbrs_model, indices, tfidf_matrix, df)
    return recommendations


indices_train = {title: idx for idx, title in enumerate(train_df['Title'])}

# Generar recomendaciones para un subconjunto de títulos
sample_titles = train_df['Title'].head(10).tolist()
recommendations = generate_recommendations(sample_titles, nbrs_train, indices_train, tfidf_matrix_train, train_df)



La métrica de cohesion calcula la distancia promedio entre las representaciones TF-IDF de los ítems recomendados para cada ítem en la lista. Esto pueda dar una idea de cuán similares son los ítems recomendados.

In [ ]:
def cohesion(recommendations, tfidf_matrix, df):
    cohesion_scores = []
    for title in recommendations:
        recommended_titles = recommendations[title]
        if isinstance(recommended_titles, np.ndarray):  # Validar que no sea un mensaje de error
            indices = [df[df['Title'] == t].index[0] for t in recommended_titles]
            neighbor_vectors = tfidf_matrix[indices]
            distances = pairwise_distances(neighbor_vectors, metric='cosine')
            upper_triangle_indices = np.triu_indices_from(distances, k=1)
            cohesion_scores.append(np.mean(distances[upper_triangle_indices]))
    return np.mean(cohesion_scores)

cohesion_score = cohesion(recommendations, tfidf_matrix_train, train_df)
print(f"Cohesión promedio de las recomendaciones: {cohesion_score}")


Cohesión promedio de las recomendaciones: 0.47447170839970926


La cohesión promedio de 0.47447170839970926 sugiere que las recomendaciones tienen una similaridad moderada entre ellas. Como el objetivo es que las recomendaciones sean muy similares al titulo original, entonces se debe aumentar la cohesión. Como no puedo disminuir la cantidad de vecinos porque se requieren de 3 titulos de libros una posible solución sea ajustar el preprocesamiento de los datos.


In [ ]:
def catalog_coverage(recommendations, df):
    recommended_items = set()
    for title in recommendations:
        recommended_titles = recommendations[title]
        if isinstance(recommended_titles, np.ndarray):  # Validar que no sea un mensaje de error
            recommended_items.update(recommended_titles)
    return len(recommended_items) / df.shape[0]

coverage_score = catalog_coverage(recommendations, train_df)
print(f"Coverage del catálogo: {coverage_score}")

Coverage del catálogo: 0.003


Un coverage del catálogo de 0.003 indica una cobertura muy baja de los ítems en tu catálogo. Esto sugiere la necesidad de mejorar la diversidad y la relevancia de las recomendaciones, así como de explorar estrategias específicas para aumentar el coverage del catálogo y proporcionar una experiencia de recomendación más completa y satisfactoria para los usuarios.

Las métricas no muy altas pueden deberse a dos factores principales: la reducción del tamaño del dataset y la elección del algoritmo de recomendación. Una reducción considerable del tamaño del dataset puede limitar la capacidad del modelo para capturar relaciones significativas, mientras que la elección del algoritmo puede no ser la más adecuada para el conjunto de datos. Se sugiere explorar el uso de Máquinas de Soporte Vectorial (SVM) como alternativa. En resumen, la optimización del modelo y la exploración de diferentes enfoques son esenciales para mejorar el rendimiento del sistema de recomendación.